# Demonstration of using Curve registry

### Essential links:
* Source code: <https://github.com/curvefi/curve-pool-registry/tree/b17>;
* ABI: <https://github.com/curvefi/curve-pool-registry/blob/b17/deployed/2020-06-17/registry.abi>;
* Registry contract: `0x06Ce8086965234400FDecAb190B115C2C0717047`.

### Complimentary information for tests:
* Y pool: `0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51` (underlying coins: DAI/USDC/USDT/TUSD, compounding coins: yDAI/yUSDC/yUSDT/yTUSD);
* DAI: `0x6B175474E89094C44Da98b954EedeAC495271d0F`;
* USDT: `0xdAC17F958D2ee523a2206206994597C13D831ec7`;

In [1]:
# Init Brownie environment
# <https://eth-brownie.readthedocs.io/en/stable/python-package.html#accessing-the-network>
from brownie import network, Contract
network.connect('mainnet')

In [2]:
import json
with open('registry.abi', 'r') as f:
    abi = json.load(f)
registry = Contract.from_abi('CurveRegistry', '0x06Ce8086965234400FDecAb190B115C2C0717047', abi)

### Finding a pool by coins
Pools can be found for a given pair (from -> to)

In [6]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 0)

'0x52EA46506B9CC5Ef470C5bf89f17Dc28bB35D85C'

In [7]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 1)

'0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51'

In [8]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 10)  # ... eventually we hit 0x0000

'0x0000000000000000000000000000000000000000'

### Getting pool's coins
Information is given in format:
```python
[
    (compounding_coin, compounding_coin, ...),
    (underlying_coin, underlying_coin, ...),
    (compounding_coin_decimals, compounding_coin_decimals, ...),
    (underlying_coin_decimals, underlying_coin_decimals, ...)
]
```

In [10]:
registry.get_pool_coins('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

(('0x16de59092dAE5CcF4A1E6439D611fd0653f0Bd01',
  '0xd6aD7a6750A7593E092a9B218d66C0A814a3436e',
  '0x83f798e925BcD4017Eb265844FDDAbb448f1707D',
  '0x73a052500105205d34Daf004eAb301916DA8190f',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000'),
 ('0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7',
  '0x0000000000085d4780B73119b644AE5ecd22b376',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000'),
 (18, 6, 6, 18, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0))

### Pool info
Format:
```python
[
    (balance_0, balance_1, ...),
    (underlying_0, underlying_1, ...),  # Real underlying balances, will likely be 0 for compounding pools
    (*precisions_for_compounding_coins ...),
    (*precisions_for_underlying_coins ...),
    pool_token_address,
    amplification,
    fee multiplied by 1e10
]
```

In [11]:
registry.get_pool_info('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

((104735076222801233702897,
  537702368172,
  1216771905596,
  470322996492898142891398,
  0,
  0,
  0,
  0),
 (0, 0, 0, 0, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0),
 '0xdF5e0e81Dff6FAF3A7e52BA697820c5e32D806A8',
 2000,
 4000000)

With these rates, underlying_balance = balance * rate / 1e18

In [12]:
registry.get_pool_rates('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

(1017578786650421068,
 1155485443855008163,
 1024236448656036017,
 1013231624926952859,
 0,
 0,
 0,
 0)

### Estimate the most probable gas spent for exchanging two coins

In [13]:
registry.estimate_gas_used('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                           '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                           '0x6B175474E89094C44Da98b954EedeAC495271d0F')

799000

### Calculate exchange amount(s)
We're swapping USDC (6 digits) to DAI (18 digits)

In [23]:
dai_amount = registry.get_exchange_amount(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             10 ** 6)  # Dump 1 USDC
dai_amount / 1e18  # DAI has 18 digits

0.9865522299903889

How much USDC do we need to get 1 DAI?

In [24]:
usdc_amount = registry.get_input_amount(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             11)
usdc_amount / 1e6

InsufficientDataBytes: Tried to read 32 bytes.  Only got 0 bytes